<a href="https://colab.research.google.com/github/Jaidon-Smith/AI-Karaoke/blob/main/STT%20Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [169]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [170]:
!pip install tfds-nightly
!pip install pydub
!pip install tensorflow_io

In [171]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import time
import tensorflow_io as tfio

In [172]:
!git clone https://github.com/Jaidon-Smith/public_datasets.git

fatal: destination path 'public_datasets' already exists and is not an empty directory.


# Building the Model

In [173]:
!pip install --quiet tensorflow-text

import tensorflow_text as text
from tensorflow.python.platform import gfile

model_file = '/gdrive/MyDrive/datasets/hiragana_jsut.model'
model = gfile.GFile(model_file, 'rb').read()

tokenizer = text.SentencepieceTokenizer(model=model)

input_vocab_size = tokenizer.vocab_size().numpy()
print(input_vocab_size)

2000


[This tutorial](https://www.tensorflow.org/tutorials/audio/simple_audio) has some good resources.

In [174]:
import pickle
jsut_speech_lengths = pickle.load(open('/gdrive/MyDrive/datasets/jsut_speech_lengths', "rb"))
jsut_text_lengths = pickle.load(open('/gdrive/MyDrive/datasets/jsut_text_lengths', "rb"))

In [175]:
!pip install pykakasi

In [176]:
tokenizer.tokenize(tf.constant("英語が分かりますか"))

<tf.Tensor: shape=(6,), dtype=int32, numpy=array([  2,   0,   5,   0, 388, 175], dtype=int32)>

In [177]:
#def preprocess_audio(old_audio):
#  audio = old_audio/tf.int16.max
#  audio = tf.cast(audio, tf.float32)
#  audio = tfio.audio.resample(audio, 48000, 24000)
#  audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
#  return audio

In [178]:
import pykakasi
converter = pykakasi.kakasi()

In [179]:
def preprocess_text(old_text):
  text = old_text
  #text = old_text.numpy()
  #text = 
  #text = text.decode('utf-8')
  text = tokenizer.tokenize(text)
  text = tf.boolean_mask(text, text != 13305)
  #text = text.to_tensor()
  #text = tf.constant(text)
  text = tf.pad(text, paddings=[[0 , 200 - tf.shape(text)[0]]])
  return text

In [180]:
def preprocess_text_no_pad(old_text):
  text = old_text
  #text = old_text.numpy()
  #text = 
  #text = text.decode('utf-8')
  text = tokenizer.tokenize(text)
  text = tf.boolean_mask(text, text != 13305)
  #text = text.to_tensor()
  #text = tf.constant(text)
  #text = tf.pad(text, paddings=[[0 , 200 - tf.shape(text)[0]]])
  text = tf.shape(text)[0]
  return text

In [181]:
# For loop that I don't think will work
#def preprocess_text(old_text):
#  batch_size = 
#  output = 
#  text = old_text.numpy().decode('utf-8')
#  text = tokenizer.tokenize(text)
#  text = tf.constant(text)
#  text = tf.pad(text, paddings=[[0 , 2000 - tf.shape(text)[0]]])
#  return text

In [182]:
#def preprocess_text_batches(old_text):
#  return tf.map_fn(
#    preprocess_text, old_text
#)

In [183]:
#text.numpy()[0].decode('utf-8')

In [184]:
#dataset = dataset.map(lambda x: {"id": x["id"], "speech": preprocess_audio(x["speech"]), "text": x["text"]})

In [185]:
#jsut_speech_lengths = {}

#for i, data in enumerate(dataset):
  
#  jsut_speech_lengths[data['id'].ref()] = data['speech'].shape[0]
#  break

In [186]:
def preprocess_audio(old_audio):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, 48000, 24000)

  #audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
  audio = tf.signal.stft(audio, frame_length=2047, frame_step=1024)

  audio = tf.pad(audio, paddings=[[0 , 500 - tf.shape(audio)[0]], [0,0]])
  return audio

In [187]:
def preprocess_audio_no_pad(old_audio):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, 48000, 24000)

  
  #audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
  audio = tf.signal.stft(audio, frame_length=2047, frame_step=1024)

  #audio = tf.pad(audio, paddings=[[0 , 2000 - tf.shape(audio)[0]], [0,0]])
  audio = tf.shape(audio)[0]
  return audio



```python
def preprocess_audio_no_pad(old_audio):
  audio = old_audio/tf.int16.max
  audio = tf.cast(audio, tf.float32)
  audio = tfio.audio.resample(audio, 48000, 24000)
  audio = tf.signal.stft(audio, frame_length=511, frame_step=256)
  #audio = tf.pad(audio, paddings=[[0 , 2000 - tf.shape(audio)[0]], [0,0]])
  speech_lengths = tf.map_fn(lambda x: tf.shape(x)[0], audio, fn_output_signature=tf.int32)
  return speech_lengths
```



In [188]:
download_config = tfds.download.DownloadConfig(manual_dir='/gdrive/MyDrive/datasets/public_datasets/downloads/manual')

dataset, info = tfds.load(
                    "jsut",
                    split="train",
                    data_dir='/gdrive/MyDrive/datasets/public_datasets',
                    download_and_prepare_kwargs={"download_config": download_config},
                    with_info = True)

In [189]:
def convert_to_kana(text):
  #kana = converter.convert('蝦夷に籠もる旧幕府軍に対する攻撃の指揮を執る。')
  new_text = text.numpy().decode('utf-8')
  kana = converter.convert(new_text)
  return ''.join([i['hira'] for i in kana])

In [190]:
dataset = dataset.map(lambda x: {"id": x["id"],
                                 "speech": x["speech"],
                                 "text": tf.py_function(convert_to_kana, [x["text"]], Tout=tf.string)})



```python
dataset = dataset.map(lambda x: {"id": x["id"],
                                 "speech_lengths": tf.py_function(func=preprocess_audio_no_pad, inp=[x["speech"]],Tout=tf.int32),
                                 "speech": preprocess_audio(x["speech"]),
                                 "text": preprocess_text(x["text"])})
```



In [191]:
dataset = dataset.map(lambda x: {"id": x["id"],
                                 "speech": preprocess_audio(x["speech"]),
                                 "speech_lengths": preprocess_audio_no_pad(x["speech"]),
                                 "text": tf.py_function(preprocess_text, [x["text"]], Tout=tf.int32),
                                 "text_lengths": tf.py_function(preprocess_text_no_pad, [x["text"]], Tout=tf.int32)})

In [192]:
#dataset = dataset.map(lambda x: {"id": x["id"],
#                                 "speech": preprocess_audio(x["speech"]),
#                                 "speech_lengths": preprocess_audio_no_pad(x["speech"]),
#                                 "text": preprocess_text(x["text"]),
#                                 "text_lengths": preprocess_text_no_pad(x["text"])})

In [193]:
#dataset = dataset.map(lambda x: {"id": x["id"], "speech": preprocess_audio(x["speech"]), "text": x["text"]})

In [194]:
dataset = dataset.batch(8)
dataset = dataset.prefetch(tf.data.AUTOTUNE)

In [195]:
#max = 0

#for i, data in enumerate(dataset):
#  value = data['speech'].shape[0]
#  if value > max:
#    max = value
#    print("new max is:", value, "with id:", data['id'].numpy())

In [196]:
ds = dataset.take(1)

In [197]:
for i in ds:
  data = i
  break

In [198]:
print(data["speech_lengths"])

tf.Tensor([222 183  99 169  69 246 100 108], shape=(8,), dtype=int32)


In [199]:
print(tf.shape(data['text']).numpy())

[  8 200]


In [200]:
print(data['text'][0][:100])

tf.Tensor(
[1543    1   12   23  166   39    6 1440   14    1  103   67   70  318
  199   11    5  109   12  134    9   89    5    1  162  293 1933   15
  801    1  158   10  130  789   31  142   14   58   10    4  614    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0], shape=(100,), dtype=int32)


In [201]:
input_vocab_size = tokenizer.vocab_size()
print(input_vocab_size)

tf.Tensor(2000, shape=(), dtype=int32)


In [202]:
speech = data['speech']
text = data['text']
id = data['id']
speech_lengths = data['speech_lengths']
text_lengths = data['text_lengths']

In [203]:
text_lengths

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([42, 34, 21, 31, 14, 56, 23, 15], dtype=int32)>

In [204]:
speech_lengths

<tf.Tensor: shape=(8,), dtype=int32, numpy=array([222, 183,  99, 169,  69, 246, 100, 108], dtype=int32)>

In [205]:
tokenizer.detokenize(text.numpy()[1]).numpy().decode('utf-8')

'あおむけはねいすかとこ、あるいはすぐにたたみによこたわれるので、しょしんしゃがれんしゅうするのにもっともらくなしせいである ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇  ⁇ '

# Using Transformer Encoder Blocks

In [206]:
d_model = 512

In [207]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)

In [208]:
def scaled_dot_product_attention(q, k, v):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  
  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    
  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
  
  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)


  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [209]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model
    
    assert d_model % self.num_heads == 0
    
    self.depth = d_model // self.num_heads
    
    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)
    
    self.dense = tf.keras.layers.Dense(d_model)
        
  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])
    
  def call(self, v, k, q):
    batch_size = tf.shape(q)[0]
    
    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)
    
    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)
    
    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v)
    
    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention, 
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)
        
    return output, attention_weights

In [210]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [211]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    
  def call(self, x, training):

    attn_output, _ = self.mha(x, x, x)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)
    
    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)
    
    return out2

In [212]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers
    
    #self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.embedding = tf.keras.layers.Dense(d_model)

    self.pos_encoding = positional_encoding(maximum_position_encoding, 
                                            self.d_model)
    
    
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) 
                       for _ in range(num_layers)]
  
    self.dropout = tf.keras.layers.Dropout(rate)
        
  def call(self, x, training=True):

    seq_len = tf.shape(x)[1]
    
    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32)) # Makes input larger by a constant
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)
    
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training)
    
    return x  # (batch_size, input_seq_len, d_model)

In [213]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [214]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, 
                                     epsilon=1e-9)

In [215]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

In [216]:
transformer = tf.keras.models.Sequential([
 
  Encoder(num_layers=6,
          d_model=d_model,
          num_heads=8,
          dff=2048, 
          maximum_position_encoding=10000),
 
  tf.keras.layers.Dense(input_vocab_size)
])

In [226]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [218]:
transformer(speech)

<tf.Tensor: shape=(8, 500, 2000), dtype=float32, numpy=
array([[[ 0.41312456,  0.17335455,  0.21648145, ..., -0.6306017 ,
         -1.0144137 , -0.55112725],
        [-0.36253595,  0.17980646, -0.01223516, ..., -0.45374295,
         -0.757069  , -0.04717982],
        [-0.47603858, -0.01441258, -0.14567396, ..., -0.45523688,
         -0.05523258, -0.05984747],
        ...,
        [-0.4194007 ,  0.19080126,  0.7686843 , ..., -0.2120717 ,
         -0.44347948, -0.16156998],
        [-0.1806181 , -0.12277628, -0.04217309, ..., -0.39153045,
         -0.3157879 , -0.3830186 ],
        [ 0.10293049, -0.25740805,  0.37956312, ...,  0.09314437,
         -0.36366427, -0.89920884]],

       [[-0.32760516,  0.3277429 , -0.08602348, ..., -0.78594303,
         -0.8697561 ,  0.02209514],
        [-0.04658628, -0.43103957, -0.319681  , ..., -0.8724724 ,
          0.12210236, -0.5773848 ],
        [ 0.42660972, -0.75599563, -0.10544842, ..., -0.9825375 ,
         -0.3502782 , -0.3830812 ],
        ...

In [227]:
@tf.function
def train_step(speech, text, speech_lengths, text_lengths):
  with tf.GradientTape() as tape:
    logits = transformer(speech)

    labels = text
    label_length = text_lengths
    logit_length = speech_lengths
    unique = tf.nn.ctc_unique_labels(text)


    loss = tf.nn.ctc_loss(
        labels=labels, logits=logits, label_length=label_length, logit_length=logit_length, logits_time_major=False, unique=unique,
    )

  gradients = tape.gradient(loss, transformer.trainable_variables)    
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)

In [228]:
EPOCHS = 5

In [229]:
for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()

  # inp -> portuguese, tar -> english
  for (batch, data) in enumerate(dataset):
    #print(batch)
    #if batch == 9:
    #  print(tokenizer.detokenize(data['text']).numpy()[0].decode('utf-8'))
    #  print(data['text'].numpy()[0])
    #  continue

    train_step(data['speech'], data['text'], data['speech_lengths'], data['text_lengths'])

    if batch % 1 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f}')
    if (batch + 1) % 20 == 0:
      ckpt_save_path = ckpt_manager.save()
      print (f'Saving checkpoint for batch {batch} at {ckpt_save_path}')

  break
  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print (f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')


Epoch 1 Batch 0 Loss 214.4406
Epoch 1 Batch 1 Loss 178.4383
Epoch 1 Batch 2 Loss 164.2824
Epoch 1 Batch 3 Loss 177.2829
Epoch 1 Batch 4 Loss 168.9137
Epoch 1 Batch 5 Loss 169.8562
Epoch 1 Batch 6 Loss 168.1096
Epoch 1 Batch 7 Loss 163.2611
Epoch 1 Batch 8 Loss 163.3279
Epoch 1 Batch 9 Loss 164.6075
Epoch 1 Batch 10 Loss 163.0068
Epoch 1 Batch 11 Loss 161.0355
Epoch 1 Batch 12 Loss 159.0959
Epoch 1 Batch 13 Loss 157.0110
Epoch 1 Batch 14 Loss 155.9013
Epoch 1 Batch 15 Loss 156.8639
Epoch 1 Batch 16 Loss 155.5773
Epoch 1 Batch 17 Loss 153.8746
Epoch 1 Batch 18 Loss 153.6978
Epoch 1 Batch 19 Loss 153.3613
Saving checkpoint for batch 19 at ./checkpoints/train/ckpt-2
Epoch 1 Batch 20 Loss 151.2765
Epoch 1 Batch 21 Loss 151.8852
Epoch 1 Batch 22 Loss 149.8111
Epoch 1 Batch 23 Loss 150.9459
Epoch 1 Batch 24 Loss 149.9103
Epoch 1 Batch 25 Loss 151.8314
Epoch 1 Batch 26 Loss 152.2852
Epoch 1 Batch 27 Loss 153.4854
Epoch 1 Batch 28 Loss 153.8719
Epoch 1 Batch 29 Loss 152.6958
Epoch 1 Batch 30 Lo

KeyboardInterrupt: ignored

In [ ]:
#

## CTC Loss



```python
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')
```



In [53]:
#dense1 = tf.keras.layers.Dense(input_vocab_size)

In [54]:
dense1 = tf.keras.layers.Dense(2000)

In [55]:
dense1_speech = dense1(speech)

In [56]:
dense1_speech.shape

TensorShape([8, 500, 2000])



```python
text_lengths = tf.map_fn(lambda x: jsut_text_lengths[x.numpy().decode('utf-8')], id, fn_output_signature=tf.int32)

speech_lengths = tf.map_fn(lambda x: jsut_speech_lengths[x.numpy().decode('utf-8')], id, fn_output_signature=tf.int32)
```



In [57]:
labels = text
logits = dense1_speech
label_length = text_lengths
logit_length = speech_lengths
unique = tf.nn.ctc_unique_labels(text)


loss = tf.nn.ctc_loss(
    labels=labels, logits=logits, label_length=label_length, logit_length=logit_length, logits_time_major=False, unique=unique,
)

Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.


Instructions for updating:
Prefer tf.tensor_scatter_nd_add, which offers the same functionality with well-defined read-write semantics.


Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.


Instructions for updating:
Prefer tf.tensor_scatter_nd_update, which offers the same functionality with well-defined read-write semantics.


In [220]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15231431426790692306, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14674281152
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14610403557497353506
 physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"]

In [176]:
import pykakasi
converter = pykakasi.kakasi()

In [175]:
kana = converter.convert('蝦夷に籠もる旧幕府軍に対する攻撃の指揮を執る。')
''.join([i['hira'] for i in kana])

'えぞにこもるきゅうばくふぐんにたいするこうげきのしきをとる。'

In [169]:
kks.convert('蝦夷に籠もる旧幕府軍に対する攻撃の指揮を執る。')

[{'hepburn': 'ezo',
  'hira': 'えぞ',
  'kana': 'エゾ',
  'kunrei': 'ezo',
  'orig': '蝦夷',
  'passport': 'ezo'},
 {'hepburn': 'ni',
  'hira': 'に',
  'kana': 'ニ',
  'kunrei': 'ni',
  'orig': 'に',
  'passport': 'ni'},
 {'hepburn': 'komo',
  'hira': 'こも',
  'kana': 'コモ',
  'kunrei': 'komo',
  'orig': '籠も',
  'passport': 'komo'},
 {'hepburn': 'ru',
  'hira': 'る',
  'kana': 'ル',
  'kunrei': 'ru',
  'orig': 'る',
  'passport': 'ru'},
 {'hepburn': 'kyuu',
  'hira': 'きゅう',
  'kana': 'キュウ',
  'kunrei': 'kyuu',
  'orig': '旧',
  'passport': 'kyuu'},
 {'hepburn': 'bakufu',
  'hira': 'ばくふ',
  'kana': 'バクフ',
  'kunrei': 'bakufu',
  'orig': '幕府',
  'passport': 'bakufu'},
 {'hepburn': 'gun',
  'hira': 'ぐん',
  'kana': 'グン',
  'kunrei': 'gun',
  'orig': '軍',
  'passport': 'gun'},
 {'hepburn': 'ni',
  'hira': 'に',
  'kana': 'ニ',
  'kunrei': 'ni',
  'orig': 'に',
  'passport': 'ni'},
 {'hepburn': 'taisuru',
  'hira': 'たいする',
  'kana': 'タイスル',
  'kunrei': 'taisuru',
  'orig': '対する',
  'passport': 'taisuru'},
 {'